In [7]:
from pathlib import Path

list((p, p.is_dir(), len(p.parts)) for p in Path("linux-5.10.10/").rglob(""))[:10]

[(PosixPath('linux-5.10.10'), True, 1),
 (PosixPath('linux-5.10.10/.ipynb_checkpoints'), True, 2),
 (PosixPath('linux-5.10.10/drivers'), True, 2),
 (PosixPath('linux-5.10.10/drivers/uio'), True, 3),
 (PosixPath('linux-5.10.10/drivers/sh'), True, 3),
 (PosixPath('linux-5.10.10/drivers/sh/superhyway'), True, 4),
 (PosixPath('linux-5.10.10/drivers/sh/maple'), True, 4),
 (PosixPath('linux-5.10.10/drivers/sh/clk'), True, 4),
 (PosixPath('linux-5.10.10/drivers/sh/intc'), True, 4),
 (PosixPath('linux-5.10.10/drivers/ssb'), True, 3)]

Looking at drivers/sh/intc/access.c  
intc_get_field_from_handle

In [1]:
target1 = "cregit_functions.file = 'drivers/sh/intc/access.c'"
target2 = "cregit_functions.file LIKE 'arch/alpha/%'"
full_kernel = "1=1"

In [2]:
def sql(query):
    !sqlite3 function_survey/output/function_survey.db "{query}"

In [13]:
sql("SELECT * FROM cregit_calls where callee = 'intc_get_field_from_handle';")
# functions calling this function, they are all tests

drivers/sh/intc/access.c|test_8|intc_get_field_from_handle
drivers/sh/intc/access.c|test_16|intc_get_field_from_handle
drivers/sh/intc/access.c|test_32|intc_get_field_from_handle


In [14]:
sql("SELECT * FROM cregit_calls where caller = 'intc_get_field_from_handle' ")
# probably indirectly tested

drivers/sh/intc/access.c|intc_get_field_from_handle|_INTC_WIDTH
drivers/sh/intc/access.c|intc_get_field_from_handle|_INTC_SHIFT


In [23]:
sql("SELECT * FROM cregit_functions where name= '_INTC_WIDTH' ")
print("***")
sql("SELECT * FROM cregit_macros where name= '_INTC_WIDTH' ")
print("***")
!grep -r _INTC_WIDTH linux-5.10.10/

***
***
linux-5.10.10/drivers/sh/intc/internals.h:#define _INTC_WIDTH(h)		((h >> 5) & 0xf)
linux-5.10.10/drivers/sh/intc/chip.c:		if (prio >= (1 << _INTC_WIDTH(ihp->handle)))
linux-5.10.10/drivers/sh/intc/chip.c:		if (value >= (1 << _INTC_WIDTH(ihp->handle)))
linux-5.10.10/drivers/sh/intc/access.c:	unsigned int width = _INTC_WIDTH(handle);
linux-5.10.10/drivers/sh/intc/access.c:	unsigned int width = _INTC_WIDTH(handle);
linux-5.10.10/drivers/sh/intc/access.c:	return fn(addr, handle, ((1 << _INTC_WIDTH(handle)) - 1));
linux-5.10.10/cscope.out:	`_INTC_WIDTH
linux-5.10.10/cscope.out:	`_INTC_WIDTH
linux-5.10.10/cscope.out:	`_INTC_WIDTH
linux-5.10.10/cscope.out:	`_INTC_WIDTH
linux-5.10.10/cscope.out:	`_INTC_WIDTH
linux-5.10.10/cscope.out:	#_INTC_WIDTH


Since `_INTC_WIDTH` is defined in a header file, we are not currently picking it up.

In [16]:
# list all tests calling 
sql(f"""EXPLAIN QUERY PLAN
SELECT cregit_functions.file AS callee_file, caller, callee,
    cregit_calls.file AS caller_file
FROM cregit_functions
INNER JOIN 
    cregit_calls
ON cregit_calls.callee = name 
    AND {target1}
AND caller LIKE '%test%'
ORDER BY caller
;""")

QUERY PLAN
|--SCAN TABLE cregit_calls
|--SEARCH TABLE cregit_functions USING COVERING INDEX idx_functions (file=? AND name=?)
`--USE TEMP B-TREE FOR ORDER BY


In [15]:
# count the number of tests affecting functions in a given file, (optionally including untested functions)
sql(f"""SELECT * FROM (
SELECT cregit_functions.file AS callee_file, name, count(function) AS num_tests
FROM cregit_functions
LEFT JOIN 
    cregit_identifiers
ON cregit_identifiers.identifier = name 
    AND cregit_identifiers.function LIKE '%test%'
WHERE
    {full_kernel}
GROUP BY callee_file, name
HAVING num_tests > 0
ORDER BY num_tests DESC)
;""")
# note that memset gets 2 different counts of the number of tests 762 and 381
# this is probably due to the microblaze file defining memset twice (762 = 2*381)

crypto/async_tx/raid6test.c|test|2596
lib/globtest.c|test|2596
samples/bpf/syscall_tp_user.c|test|2596
tools/perf/tests/expr.c|test|2596
tools/perf/tests/kmod-path.c|test|2596
tools/testing/selftests/bpf/prog_tests/sock_fields.c|test|2596
tools/testing/selftests/kvm/x86_64/mmio_warning_test.c|test|2596
tools/testing/selftests/net/reuseport_addr_any.c|test|2596
tools/testing/selftests/net/reuseport_bpf_cpu.c|test|2596
tools/testing/selftests/net/reuseport_bpf_numa.c|test|2596
tools/testing/selftests/net/reuseport_dualstack.c|test|2596
tools/testing/selftests/powerpc/mm/bad_accesses.c|test|2596
tools/testing/selftests/powerpc/mm/large_vm_fork_separation.c|test|2596
tools/testing/selftests/powerpc/mm/pkey_exec_prot.c|test|2596
tools/testing/selftests/powerpc/mm/pkey_siginfo.c|test|2596
tools/testing/selftests/powerpc/mm/stack_expansion_ldst.c|test|2596
tools/testing/selftests/powerpc/mm/stack_expansion_signal.c|test|2596
tools/testing/selftests/powerpc/syscalls/rtas_filter.c|test|2596
too

drivers/dma/mv_xor.c|mv_xor_alloc_chan_resources|2
drivers/dma/mv_xor.c|mv_xor_free_chan_resources|2
drivers/dma/mv_xor.c|mv_xor_issue_pending|2
drivers/dma/mv_xor.c|mv_xor_status|2
drivers/dma/mv_xor.c|mv_xor_tx_submit|2
drivers/edac/edac_pci_sysfs.c|get_pci_parity_status|2
drivers/firmware/psci/psci_checker.c|down_and_up_cpus|2
drivers/gpu/drm/amd/amdgpu/amdgpu_object.c|amdgpu_bo_create|2
drivers/gpu/drm/amd/amdgpu/amdgpu_object.c|amdgpu_bo_gpu_offset|2
drivers/gpu/drm/amd/amdgpu/amdgpu_object.c|amdgpu_bo_pin|2
drivers/gpu/drm/amd/amdgpu/amdgpu_ttm.c|amdgpu_copy_buffer|2
drivers/gpu/drm/amd/amdgpu/atom.c|atom_get_dst|2
drivers/gpu/drm/amd/amdgpu/atom.c|atom_get_src|2
drivers/gpu/drm/amd/display/dc/core/dc_link_dp.c|set_crtc_test_pattern|2
drivers/gpu/drm/amd/display/dc/dce/dmub_hw_lock_mgr.c|dmub_hw_lock_mgr_cmd|2
drivers/gpu/drm/amd/display/dc/dce/dmub_hw_lock_mgr.c|should_use_dmub_lock|2
drivers/gpu/drm/amd/display/modules/hdcp/hdcp_psp.c|mod_hdcp_hdcp1_enable_encryption|2
drivers/

In [6]:
sql("""CREATE INDEX idx_functions ON cregit_functions (file, name)""")
# put the columsn commonly used in where/order_by clauses in the index
# function/identifier names should be in there
# 

In [5]:
sql(".schema")

CREATE VIEW used_func_names(file, outer_function, name) AS
SELECT 
  cregit_names.file AS file, cregit_names.function_name AS outer_function,
  cregit_names.token_name AS name
FROM cregit_names INNER JOIN cregit_functions ON cregit_names.token_name = cregit_functions.name;
CREATE VIEW test_funcs AS SELECT * from cregit_functions WHERE name LIKE '%test%'
/* test_funcs(file,name) */;
CREATE VIEW tested_funcs AS SELECT * FROM test_funcs JOIN cregit_calls ON cregit_calls.caller = test_funcs.name
/* tested_funcs(file,name,"file:1",caller,callee) */;
CREATE TABLE cregit_functions (file,name);
CREATE TABLE cregit_calls (file,caller,callee);
CREATE TABLE cregit_includes (file,include);
CREATE TABLE cregit_specifiers (file,name,specifier);
CREATE TABLE cregit_identifiers (file,function,identifier);
CREATE TABLE cregit_macros (file,name);


In [94]:
sql("SELECT COUNT(*) FROM cregit_functions;")

533158


In [86]:
sql("""
SELECT * FROM cregit_functions WHERE name = 'memset'
;""")

arch/sh/boot/compressed/misc.c|memset
arch/mips/boot/compressed/string.c|memset
arch/parisc/boot/compressed/misc.c|memset
arch/h8300/boot/compressed/misc.c|memset
arch/nios2/lib/memset.c|memset
arch/nios2/boot/compressed/misc.c|memset
arch/m68k/lib/memset.c|memset
arch/arm/boot/compressed/string.c|memset
arch/x86/lib/memcpy_32.c|memset
arch/x86/boot/compressed/string.c|memset
arch/microblaze/lib/memset.c|memset
arch/microblaze/lib/memset.c|memset
lib/string.c|memset
mm/kasan/common.c|memset
